In [1]:

import tensorflow as tf
import os
import io
import pandas as pd
import re

In [2]:
tf.__version__

'2.5.0'

* Đọc dữ liệu ko có định dạng rõ ràng.

In [3]:
lines = io.open("./data/smsspamcollection/SMSSpamCollection").read().strip().split('\n')

lines[0]

'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

* Tách các label và text ra, đồng thời gán giá trị $1$ cho `spam` và $0$ cho `ham`.

In [4]:
spam_dataset = []

for line in lines:
    label, text = line.split('\t')
    
    if label.strip() == 'spam':
        spam_dataset.append((1, text.strip()))
    else:
        spam_dataset.append((0, text.strip()))

In [5]:
spam_dataset[0]

(0,
 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')

In [6]:
len(spam_dataset)

5574

# Text normalization

In [7]:
df = pd.DataFrame(spam_dataset, columns=['Spam', 'Message'])

In [8]:
def message_length(x):
    return len(x)

def num_capitals(x):
    _, count = re.subn(r"[A-Z]", "", x)
    return count

def num_punctuation(x):
    _, count = re.subn(r"\W", "", x)
    return count

In [9]:
df['Capitals'] = df['Message'].apply(num_capitals)
df['Punctuation'] = df['Message'].apply(num_punctuation)
df['Length'] = df["Message"].apply(message_length)

df.describe()

,Spam,Capitals,Punctuation,Length
count,5574.000000,5574.000000,5574.000000,5574.000000
mean,0.134015,5.621636,18.942591,80.443488
std,0.340699,11.683233,14.825994,59.841746
min,0.000000,0.000000,0.000000,2.000000
25%,0.000000,1.000000,8.000000,36.000000
50%,0.000000,2.000000,15.000000,61.000000
75%,0.000000,4.000000,27.000000,122.000000
max,1.000000,129.000000,253.000000,910.000000


* Chia dữ liệu thành train data và test data với kích thước 80% và 20%.

In [10]:
train = df.sample(frac=0.8, random_state=42)
test = df.drop(train.index)

x_train = train[['Length', 'Capitals', 'Punctuation']]
y_train = train[['Spam']]

x_test = test[['Length', 'Capitals', 'Punctuation']]
y_test = test[['Spam']]

# Modeling normalized data
* Model sẽ sử dụng binary cross-entropy để tính toán loss value và Adam optimizer để train data.
* MOdel dc build dựa trên 3 feature là `Length`, `Capitals` và `Punctuation`.

In [11]:
def make_model(input_dims=3, num_units=12):
    """
    Dùng để build model

    Args:
        input_dims (int, optional): Số feature có trong input.
        num_units (int, optional): [description]. Defaults to 12.

    Returns:
        [type]: [description]
    """
    model = tf.keras.Sequential()
    
    '''Thêm một layer được kết nối với 12 unit của model'''
    model.add(tf.keras.layers.Dense(num_units,
                                    input_dim=input_dims,
                                    activation='relu'))
    
    '''Thêm một sigmoid layer với binary output unit'''
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [12]:
model = make_model()
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 1s 725us/step - loss: 0.5489 - accuracy: 0.8823
Epoch 2/10
446/446 [==============================] - 0s 921us/step - loss: 0.3178 - accuracy: 0.9054
Epoch 3/10
446/446 [==============================] - 0s 674us/step - loss: 0.2585 - accuracy: 0.9199
Epoch 4/10
446/446 [==============================] - 0s 718us/step - loss: 0.2384 - accuracy: 0.9224
Epoch 5/10
446/446 [==============================] - 0s 694us/step - loss: 0.2290 - accuracy: 0.9217
Epoch 6/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2306 - accuracy: 0.9199
Epoch 7/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2291 - accuracy: 0.9199
Epoch 8/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2156 - accuracy: 0.9242
Epoch 9/10
446/446 [==============================] - 0s 779us/step - loss: 0.2151 - accuracy: 0.9244
Epoch 10/10
446/446 [==============================] - 0s 999us/step - loss: 0.2212 - ac

In [13]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 1ms/step - loss: 0.2062 - accuracy: 0.9238


[0.20615598559379578, 0.9237667918205261]

In [14]:
y_train_pred = model.predict_classes(x_train)

/home/manhcuong/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [15]:
tf.math.confusion_matrix(tf.constant(y_train.Spam), y_train_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3781,   86],
       [ 221,  371]], dtype=int32)>

# Tokenization
* Có thể xem các ngôn ngữ mà `stanfordnlp` hỗ trợ tokenization tại [https://stanfordnlp.github.io/stanfordnlp/models.html](https://stanfordnlp.github.io/stanfordnlp/models.html).

In [17]:
import stanfordnlp as snlp

In [19]:
vi = snlp.download('vi')

Using the default treebank "vi_vtb" for language "vi".
Would you like to download the models for: vi_vtb now? (Y/n)

Default download directory: /home/manhcuong/stanfordnlp_resources
Hit enter to continue or type an alternate directory.

Download location: /home/manhcuong/stanfordnlp_resources/vi_vtb_models.zip


100%|██████████| 217M/217M [00:52<00:00, 4.13MB/s]



Download complete.  Models saved to: /home/manhcuong/stanfordnlp_resources/vi_vtb_models.zip
Extracting models file for: vi_vtb
Cleaning up...Done.


In [20]:
en = snlp.download('en')

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)

Default download directory: /home/manhcuong/stanfordnlp_resources
Hit enter to continue or type an alternate directory.

Download location: /home/manhcuong/stanfordnlp_resources/en_ewt_models.zip


100%|██████████| 235M/235M [00:57<00:00, 4.09MB/s]



Download complete.  Models saved to: /home/manhcuong/stanfordnlp_resources/en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.


In [21]:
vi = snlp.Pipeline(lang='vi', processors='tokenize')

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/manhcuong/stanfordnlp_resources/vi_vtb_models/vi_vtb_tokenizer.pt', 'lang': 'vi', 'shorthand': 'vi_vtb', 'mode': 'predict'}
Done loading processors!
---


In [27]:
sentence = "Có con đường nào bước qua ta đến mang em món quà hẹn hò yêu thương ta say đến già. Nắng mưa là chuyện nắng mưa, ai biết yêu thương đã vừa..."
tokenized = vi(sentence)

In [28]:
len(tokenized.sentences)

2

> * `sentence` gồm 2 câu, câu 1 bắt đầu từ "Có con đường... đến già.". Câu hai là "Nắng mưa... đã vừa..."

In [29]:
for snt in tokenized.sentences:
    for word in snt.tokens:
        print(word.text)
        
    print("<End of Sentence>")

Có
con
đường
nào
bước
qua ta
đến
mang
em
món
quà
hẹn
hò
yêu
thương
ta
say
đến
già
.
<End of Sentence>
Nắng
mưa
là
chuyện
nắng mưa
,
ai
biết
yêu thương
đã
vừa
...
<End of Sentence>


In [37]:
en = snlp.Pipeline(lang='en', processors='tokenize')

def word_counts(x, pipeline=en):
    """
    Dùng để đếm số từ trong `x`
    """
    doc = pipeline(x)
    count = sum([len(sentence.tokens) for sentence in doc.sentences])
    
    return count

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/manhcuong/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---


In [38]:
train['Words'] = train['Message'].apply(word_counts)
test['Words'] = test['Message'].apply(word_counts)

x_train = train[['Words', 'Capitals', 'Punctuation', 'Length']]
y_train = train[['Spam']]

x_test = test[['Words', 'Capitals', 'Punctuation', 'Length']]
y_test = test[['Spam']]

model = make_model(input_dims=4)

In [39]:
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 1s 2ms/step - loss: 0.5663 - accuracy: 0.8636
Epoch 2/10
446/446 [==============================] - 1s 2ms/step - loss: 0.3019 - accuracy: 0.9231
Epoch 3/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2640 - accuracy: 0.9291
Epoch 4/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2423 - accuracy: 0.9316
Epoch 5/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2283 - accuracy: 0.9298
Epoch 6/10
446/446 [==============================] - 1s 2ms/step - loss: 0.2145 - accuracy: 0.9312
Epoch 7/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2109 - accuracy: 0.9327
Epoch 8/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2147 - accuracy: 0.9296
Epoch 9/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2010 - accuracy: 0.9309
Epoch 10/10
446/446 [==============================] - 0s 1ms/step - loss: 0.2010 - accuracy: 0.9329

In [40]:
train.loc[train.Spam == 1].describe()

,Spam,Capitals,Punctuation,Length,Words
count,592.0,592.000000,592.000000,592.000000,592.000000
mean,1.0,15.320946,29.086149,138.856419,29.511824
std,0.0,11.635105,7.083572,28.079980,7.474256
min,1.0,0.000000,2.000000,13.000000,3.000000
25%,1.0,7.000000,26.000000,132.000000,26.000000
50%,1.0,14.000000,30.000000,149.000000,30.000000
75%,1.0,21.000000,34.000000,157.000000,35.000000
max,1.0,128.000000,49.000000,197.000000,49.000000


In [41]:
train.loc[train.Spam == 0].describe()

,Spam,Capitals,Punctuation,Length,Words
count,3867.0,3867.000000,3867.000000,3867.000000,3867.000000
mean,0.0,4.018878,17.325058,71.354538,17.344194
std,0.0,10.599291,14.826644,57.755351,13.811278
min,0.0,0.000000,0.000000,2.000000,1.000000
25%,0.0,1.000000,8.000000,33.000000,8.000000
50%,0.0,2.000000,13.000000,53.000000,13.000000
75%,0.0,3.000000,23.000000,92.000000,22.000000
max,0.0,129.000000,253.000000,910.000000,209.000000


# Stopwords

In [42]:
import stopwordsiso as stopwords

In [45]:
vi_sw = stopwords.stopwords('vi')

In [48]:
len(vi_sw)

645

In [49]:
vi_sw

{'a ha',
 'a-lô',
 'ai',
 'ai ai',
 'ai nấy',
 'alô',
 'amen',
 'anh',
 'bao giờ',
 'bao lâu',
 'bao nhiêu',
 'bao nả',
 'bay biến',
 'biết',
 'biết bao',
 'biết bao nhiêu',
 'biết chừng nào',
 'biết mấy',
 'biết đâu',
 'biết đâu chừng',
 'biết đâu đấy',
 'bà',
 'bài',
 'bác',
 'bây bẩy',
 'bây chừ',
 'bây giờ',
 'bây nhiêu',
 'bèn',
 'béng',
 'bông',
 'bạn',
 'bản',
 'bất chợt',
 'bất cứ',
 'bất giác',
 'bất kì',
 'bất kể',
 'bất kỳ',
 'bất luận',
 'bất nhược',
 'bất quá',
 'bất thình lình',
 'bất tử',
 'bất đồ',
 'bấy',
 'bấy chầy',
 'bấy chừ',
 'bấy giờ',
 'bấy lâu',
 'bấy lâu nay',
 'bấy nay',
 'bấy nhiêu',
 'bập bà bập bõm',
 'bập bõm',
 'bắt đầu từ',
 'bằng',
 'bằng không',
 'bằng nấy',
 'bằng ấy',
 'bển',
 'bệt',
 'bị',
 'bỏ mẹ',
 'bỗng',
 'bỗng chốc',
 'bỗng dưng',
 'bỗng không',
 'bỗng nhiên',
 'bỗng đâu',
 'bộ',
 'bội phần',
 'bớ',
 'bởi',
 'bởi chưng',
 'bởi nhưng',
 'bởi thế',
 'bởi vì',
 'bởi vậy',
 'bức',
 'cao',
 'cha',
 'cha chả',
 'chao ôi',
 'chiếc',
 'cho',
 'cho nên

* Cập nhật lại hàm `word_counts` thêm chức năng loại bỏ stopword như sau:

In [50]:
en_sw = stopwords.stopwords('en')
def word_counts(x, pipeline=en):
    doc = pipeline(x)
    cnt = 0
    
    for sentence in doc.sentences:
        for token in sentence.tokens:
            if token.text.lower() not in en_sw:
                cnt += 1
                
    return cnt

In [51]:
train['Words'] = train['Message'].apply(word_counts)
test['Words'] = test['Message'].apply(word_counts)

x_train = train[['Words', 'Capitals', 'Punctuation', 'Length']]
y_train = train[['Spam']]

x_test = test[['Words', 'Capitals', 'Punctuation', 'Length']]
y_test = test[['Spam']]

model = make_model(input_dims=4)

In [52]:
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
446/446 [==============================] - 1s 1ms/step - loss: 0.6630 - accuracy: 0.8210
Epoch 2/10
446/446 [==============================] - 1s 1ms/step - loss: 0.4092 - accuracy: 0.8713
Epoch 3/10
446/446 [==============================] - 1s 1ms/step - loss: 0.3066 - accuracy: 0.9087
Epoch 4/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2497 - accuracy: 0.9258
Epoch 5/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2196 - accuracy: 0.9300
Epoch 6/10
446/446 [==============================] - 1s 1ms/step - loss: 0.2034 - accuracy: 0.9368
Epoch 7/10
446/446 [==============================] - 1s 1ms/step - loss: 0.1928 - accuracy: 0.9368
Epoch 8/10
446/446 [==============================] - 1s 1ms/step - loss: 0.1832 - accuracy: 0.9394
Epoch 9/10
446/446 [==============================] - 1s 2ms/step - loss: 0.1816 - accuracy: 0.9374
Epoch 10/10
446/446 [==============================] - 1s 1ms/step - loss: 0.1737 - accuracy: 0.9433

In [53]:
train.loc[train.Spam == 1].describe()

,Spam,Capitals,Punctuation,Length,Words
count,592.0,592.000000,592.000000,592.000000,592.000000
mean,1.0,15.320946,29.086149,138.856419,18.464527
std,0.0,11.635105,7.083572,28.079980,6.100852
min,1.0,0.000000,2.000000,13.000000,2.000000
25%,1.0,7.000000,26.000000,132.000000,14.000000
50%,1.0,14.000000,30.000000,149.000000,19.000000
75%,1.0,21.000000,34.000000,157.000000,23.000000
max,1.0,128.000000,49.000000,197.000000,33.000000


In [54]:
train.loc[train.Spam == 0].describe()

,Spam,Capitals,Punctuation,Length,Words
count,3867.0,3867.000000,3867.000000,3867.000000,3867.000000
mean,0.0,4.018878,17.325058,71.354538,7.911042
std,0.0,10.599291,14.826644,57.755351,7.326390
min,0.0,0.000000,0.000000,2.000000,0.000000
25%,0.0,1.000000,8.000000,33.000000,4.000000
50%,0.0,2.000000,13.000000,53.000000,6.000000
75%,0.0,3.000000,23.000000,92.000000,10.000000
max,0.0,129.000000,253.000000,910.000000,147.000000


In [55]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 3ms/step - loss: 0.2071 - accuracy: 0.9300


[0.20706488192081451, 0.9300448298454285]